In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
from sklearn.metrics import mean_squared_error
import lasagne as ls
from theano import tensor as T
from lasagne.layers import InputLayer, DenseLayer
from lasagne.updates import nesterov_momentum
from lasagne.nonlinearities import rectify
from nolearn.lasagne import NeuralNet
import sqlite3
# from sklearn import mean_squared_error
from pandas import HDFStore
store = HDFStore("storeTraffic.h5")
conn = sqlite3.connect('trafficdb')

In [13]:
class LoadParam():
    def initNN(self,n_input):
        #Build layer for MLP
        l_in = ls.layers.InputLayer(shape=(None,n_input),input_var=None)
        l_hidden = ls.layers.DenseLayer(l_in,num_units=15,nonlinearity=ls.nonlinearities.rectify)
        network = l_out = ls.layers.DenseLayer(l_hidden,num_units=1)
        print "Neural network initialize"
        #Init Neural net
        net1 = NeuralNet(
            layers=network,
            # optimization method:
            update=nesterov_momentum,
            update_learning_rate=0.000001,
            update_momentum=0.9,
            regression=True,  # flag to indicate we're dealing with regression problem
            max_epochs=400,  # we want to train this many epochs
            verbose=1,
        )
        return net1
    def __init__(self,n_input,type="NN"):
        self.n_input = n_input
        self.net = self.initNN(n_input)
        if(type=="NN"):
            self.net.load_params_from('Params/saveNeuralNetwork_1e-05_%s.tdn'%n_input)
        elif(type=="GN"):
            self.net.load_params_from('GeneticParams/saveNeuralNetwork_1e-05_%s.tdn'%n_input)
    
    def normalize(self,dataCount,dataTest):
        dataNorm = pd.Series(np.zeros(dataCount.shape[0]),dtype=np.float64)
        dataNorm = (dataCount - dataTest.min())/(dataTest.max()-dataTest.min())
        return dataNorm
    def convert(self,data,dataTest):
        max = dataTest.max()
        min = dataTest.min()
        return (data*(max-min)+min)
    def predict(self,dataTest):
#         dataTest= pd.read_sql("SELECT count FROM workload where time >= 895096802-%d and time < 895096802+86400"%(n_input),conn)["count"]
        range_list = len(dataTest)
        jump_list = 600
        init_list = 0;
        length = range_list/jump_list
        dataCount = np.array(np.zeros(length))
        print "Count list"
        for i in np.arange(0,length):
            tmp = dataTest[init_list:init_list+jump_list]
            dataCount[i] = tmp.sum()
            init_list+=jump_list
        n_row = dataCount.shape[0]
        self.X_training = self.normalize(np.asarray([[dataCount[t-i-1] for i in range(0,self.n_input)]
                         for t in np.arange(self.n_input,n_row)]),store["raw_conn_train"])
        print "y_training loading..."
        self.y_training = np.asarray(dataCount[self.n_input:n_row])
        self.result = self.net.predict(self.X_training)
        return self.convert(self.result,store["raw_conn_train"])

In [14]:
NN = LoadParam(7,"NN")
GN = LoadParam(10,"GN")

Neural network initialize
Loaded parameters to layer 'dense1' (shape 7x15).
Loaded parameters to layer 'dense1' (shape 15).
Loaded parameters to layer 'dense2' (shape 15x1).
Loaded parameters to layer 'dense2' (shape 1).
Neural network initialize
Loaded parameters to layer 'dense1' (shape 10x15).
Loaded parameters to layer 'dense1' (shape 15).
Loaded parameters to layer 'dense2' (shape 15x1).
Loaded parameters to layer 'dense2' (shape 1).


In [15]:
dataTestNN= pd.read_sql("SELECT count FROM workload where time >= 895096802-%d and time < 895096802+86400"%(NN.n_input),conn)["count"]
dataTestGN= pd.read_sql("SELECT count FROM workload where time >= 895096802-%d and time < 895096802+86400"%(GN.n_input),conn)["count"]
    

In [19]:
nn_pred = NN.predict(dataTestNN)
gn_pred = GN.predict(dataTestGN)
nn_rmse = NN.net.score(NN.X_training,NN.y_training)
gn_rmse = GN.net.score(GN.X_training,GN.y_training)

Count list
y_training loading...
Count list
y_training loading...


In [18]:
f,(ax1,ax2) = pl.subplots(2,sharex=True,sharey=True)
ax1.plot(gn_pred,label="Genetic Neural Network")
ax1.set_title("Genetic Neural Network")
ax1.plot(GN.y_training)
# ax1.set_color_cycle(['blue','red'])
ax2.plot(nn_pred,label="Neural Network")
ax2.plot(NN.y_training)
ax2.set_title("Neural Network")
# ax2.set_color_cycle(['blue','red'])
pl.show()

In [20]:
nn_rmse, gn_rmse

(196632324.83461443, 199208095.32522783)